# Loading of the dataset and First preprocessing step
for this project I chose the dataset available on Hugging face ncbi-virus-complete-dna-v230722, but reducing its size, the original dataset contains over 2 million rows, but for a matter of complexity and capacity of the pc I took a portion containing 200 000 items, 

To download the complete dataset, you can run the cell below which will automatically check if the dataset is in the folder, otherwise the download

In [37]:
import json
import glob

import numpy as np
from datasets import load_dataset
import pandas as pd
import os
from imblearn.over_sampling import SMOTEN

path_multiclass = 'Dataset/MulticlassDatasets'
path_dataframe = 'Dataset/ncbi-virus-200-000-nucleotides'

To reduce the dataset size we take the first 200 000 rows of the dataset

In [2]:
# number of rows selected
n = 200000
if not glob.glob(path_dataframe+'.*'):
    print('Downloading dataset...')
    # load dataset from hugging face
    dataset = load_dataset("LKarlo/ncbi-virus-complete-dna-v230722")
    # Save the dataset on parquet extention
    dataset = dataset['train'].iloc[:n]
    dataset.to_parquet(path_dataframe + '.parquet')
else: 
    print('Dataset already exists')

Dataset already exists


In [3]:
if not os.path.exists(path_dataframe+'.parquet'):
    print('Loading dataset from file jsonl...')
    with open(path_dataframe, 'rb') as f:
        df = None
        for line in f:
            jo = json.loads(line)
            if df is None:
                print('Parsing dataset...')
                df = pd.DataFrame(columns=jo.keys())
            
            df.loc[len(df)] = jo.values()
        # Save parquet file in local
        df.to_parquet(path_dataframe+'.parquet', index=False)
        print('Done!')
else:
    print('Loading dataset from parquet...')
    df = pd.read_parquet(path_dataframe + '.parquet')
    
df.head()

Loading dataset from parquet...


,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_079174.1,2023-05-17T00:00:00Z,Erwinia phage Fifi44,Loessnervirus,Chaseviridae,dsDNA,53559,RefSeq,Erwinia amylovora,2020-09,TGTTCCATGTGAGTTACTGAGTTCCGTAATCCTATTATAAAGGCCC...
1,NC_079175.1,2023-05-17T00:00:00Z,Escherichia phage PO103-1,Carltongylesvirus,Chaseviridae,dsDNA,53040,RefSeq,Escherichia coli,nan,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATAAATAAAATTA...
2,NC_079176.1,2023-05-17T00:00:00Z,Escherichia phage MLP1,Carltongylesvirus,Chaseviridae,dsDNA,51940,RefSeq,Escherichia coli CFT073,2019,GCTGCTGCTGCCAGCTTAGCACAGGAAGCCACCATTGATGTGGTAA...
3,NC_079177.1,2023-05-17T00:00:00Z,Yersinia phage PYps3T,Carltongylesvirus,Chaseviridae,dsDNA,52854,RefSeq,Yersinia pseudotuberculosis,2019,CGTGAATTGTGTAATGGCTACGCTATTATAATCCGTTTCATTCAAA...
4,NC_079178.1,2023-05-17T00:00:00Z,Yersinia phage PYps23T,Carltongylesvirus,Chaseviridae,dsDNA,54869,RefSeq,Yersinia pseudotuberculosis,2019,CGTGAATTGTGTAATGGCTACGCTATTATAATCCGTTTCATTCAAA...


# Multiclass classification

## For the first task we analize the distribution of the feature target

In [4]:
df['Family'].value_counts()

Family
Coronaviridae       180522
nan                   5328
Geminiviridae          850
Spinareoviridae        545
Phenuiviridae          495
                     ...  
Helgolandviridae         1
Aggregaviridae           1
Redondoviridae           1
Myriaviridae             1
Plasmaviridae            1
Name: count, Length: 233, dtype: int64

Let’s delete the rows that contain nan to the family

In [5]:
index = df[df['Family'] == 'nan'].index
df.drop(index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_079174.1,2023-05-17T00:00:00Z,Erwinia phage Fifi44,Loessnervirus,Chaseviridae,dsDNA,53559,RefSeq,Erwinia amylovora,2020-09,TGTTCCATGTGAGTTACTGAGTTCCGTAATCCTATTATAAAGGCCC...
1,NC_079175.1,2023-05-17T00:00:00Z,Escherichia phage PO103-1,Carltongylesvirus,Chaseviridae,dsDNA,53040,RefSeq,Escherichia coli,nan,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATAAATAAAATTA...
2,NC_079176.1,2023-05-17T00:00:00Z,Escherichia phage MLP1,Carltongylesvirus,Chaseviridae,dsDNA,51940,RefSeq,Escherichia coli CFT073,2019,GCTGCTGCTGCCAGCTTAGCACAGGAAGCCACCATTGATGTGGTAA...
3,NC_079177.1,2023-05-17T00:00:00Z,Yersinia phage PYps3T,Carltongylesvirus,Chaseviridae,dsDNA,52854,RefSeq,Yersinia pseudotuberculosis,2019,CGTGAATTGTGTAATGGCTACGCTATTATAATCCGTTTCATTCAAA...
4,NC_079178.1,2023-05-17T00:00:00Z,Yersinia phage PYps23T,Carltongylesvirus,Chaseviridae,dsDNA,54869,RefSeq,Yersinia pseudotuberculosis,2019,CGTGAATTGTGTAATGGCTACGCTATTATAATCCGTTTCATTCAAA...


In [6]:
df['Family'].value_counts()

Family
Coronaviridae       180522
Geminiviridae          850
Spinareoviridae        545
Phenuiviridae          495
Sedoreoviridae         469
                     ...  
Helgolandviridae         1
Aggregaviridae           1
Redondoviridae           1
Myriaviridae             1
Plasmaviridae            1
Name: count, Length: 232, dtype: int64

### For the multiclass we select the 4 most frequent classes
We select Coronaviridae, Geminiviridae, Spinareoviridae and Phenuiviridae.

We save the dataset both to make an unbalanced classification and to oversample the dataset.

In [41]:
selected_df = df.query(
    'Family == "Coronaviridae" or'
    ' Family == "Geminiviridae" or'
    ' Family == "Spinareoviridae" or'
    ' Family == "Phenuiviridae"'
)

In [8]:
selected_df['Family'].value_counts()

Family
Coronaviridae      180522
Geminiviridae         850
Spinareoviridae       545
Phenuiviridae         495
Name: count, dtype: int64

We check if the dataset has already been saved and save it

In [10]:
if not os.path.exists(path_multiclass+'/UnbalancedDataset.parquet'):
    print('Saving the dataset...')
    selected_df.to_parquet(path_multiclass + '/UnbalancedDataset.parquet')
    print('Done!')
else:
    print('Dataset already exists')

Dataset already exists


We perform the oversampling with the SMOTEN technique.

The SMOTEN technique is an oversampling technique of minority classes that also works with categorical variables, the imblearn library containing its implementation is used, the reference paper talks about the SMOTE technique in general.

The reference link is the above: [SMOTEN](https://doi.org/10.48550/arXiv.1106.1813) 

In [38]:
cols = selected_df.columns.values.tolist()
cols.remove('Family')
y = selected_df['Family']
X = selected_df[cols] 
resampler = SMOTEN(random_state=42)

X,y = resampler.fit_resample(X,y)

In [39]:
X['Family'] = y

X['Family'].value_counts()

Family
Geminiviridae      180522
Spinareoviridae    180522
Phenuiviridae      180522
Coronaviridae      180522
Name: count, dtype: int64

In [40]:
if not os.path.exists(path_multiclass+'/BalancedDataset.parquet'):
    print('Saving the dataset...')
    X.to_parquet(path_multiclass + '/BalancedDataset.parquet')
    print('Done!')
else:
    print('Dataset already exists')

Dataset already exists
